<a href="https://colab.research.google.com/github/snowyTheHamster/autoImageRenameCropResize/blob/master/images_remove_bg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Open CV Auto Crop Images

Remove Background from Object in images (works on white background).

---

### How to use
**Prepare folders**

+ Create a project folder in Google Drive.
+ Create a folder inside project folder with images you want to crop.
+ Create another folder inside project folder to output results.

**Adjust variables and settings in step 2**

+ Default settings should work fine but you can adjust the parameters to finetune results.

+ You can also change some options in step 2 for the kernel, threshold and morph choices.

+ If you still can't get good results, prepare photos with cleaner white backgrounds with better distinction between object and background.

+ If the object in white, the script may have trouble distinguishing it from background.

+ If the object is black, the script may have trouble distinguishing it from the shadows.

**Run Code**

+ Run each of the blocks in order 1 ~ 4.


### 1. Mount Google Drive

In [1]:
from google.colab import drive
import os
import cv2
import numpy as np

drive.mount('/content/drive/')

Mounted at /content/drive/


### 2. Adjust Settings Below

In [2]:
# Make sure folder names match what you created in google drive
project_folder = 'images_test'
input_folder = 'images_start'
output_folder = 'images_bg_removed'

# define image file formats here
image_exts = [ '.jpg', '.jpeg' ]

# Threshold Values
MIN_VALUE = 200  # default: 200
MAX_VALUE = 10   # default: 10
MASK_VALUE = 255 # default: 255

# Background Fill Value
FILL_1 = 255 # default: 255
FILL_2 = 255 # default: 255
FILL_3 = 255 # default: 255

# 1st iteration parameters
## bilateralFilter values
a_bf0 = 6   # default: 6
a_bf1 = 231 # default: 231
a_bf2 = 231 # default: 231

## Canny blurred values
a_cb0 = 2
a_cb1 = 6

## getStructuringElement values
a_se0 = 77
a_se1 = 77

# 2nd iteration parameters
## bilateralFilter values
b_bf0 = 6   # default: 6
b_bf1 = 131 # default: 131
b_bf2 = 131 # default: 131

## getStructuringElement values
b_se0 = 9
b_se1 = 9


# dont need to change these
work_dir = os.path.join('/content/drive/My Drive/', project_folder)
dir_to_work = os.path.join(work_dir, input_folder)
dir_imgs_results = os.path.join(work_dir, output_folder)
FILL_COLOR = [FILL_1, FILL_2, FILL_3]

### 3. Run Script

In [3]:
count = 0
for _, _, files in os.walk(dir_to_work):
  file_count = len(files)
  for filename in files:
    count += 1
    file_path = os.path.join(dir_to_work, filename)
    file_name, file_ext = os.path.splitext(file_path)
    output_file_name = os.path.basename(file_name) + file_ext

    if file_ext not in image_exts:
      print("skipping " + filename + " (not a supported image file format)")
      count -= 1
      file_count -= 1
      continue

    else:
      print(f'{count}/{file_count} Removing BG from {filename}...')

      # FIRST ITERATION OF THE IMAGE PROCESS
      image = cv2.imread(file_path)
      gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
      blurred = cv2.bilateralFilter(gray, a_bf0, a_bf1, a_bf2)

      #PLAN B: Canny
      thresh = cv2.Canny(blurred, a_cb0, a_cb1)

      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (a_se0, a_se1)) # available options: MORPH_RECT, MORPH_ELLIPSE; Also tweak last parameter(x, x)
      morphchoice = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel) # available options: MORPH_CLOSE, MORPH_OPEN, MORPH_DILATE, MORPH_ERODE
      (cnts, _) = cv2.findContours(morphchoice.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # finding_contours
      
      #find the biggest contour
      c = max(cnts, key = cv2.contourArea)
      
      #place c back in a list
      list_c = [ np.array( c ) ]

      #FILL OUTSIDE OF COUTOUR WITH COLOR
      stencil = np.zeros(image.shape[:-1]).astype(np.uint8)
      cv2.fillPoly(stencil, list_c, MASK_VALUE)
      sel = stencil != MASK_VALUE # select everything that is not MASK_VALUE
      image[sel] = FILL_COLOR # and fill it with FILL_COLOR

      ###
      # SECOND ITERATION OF THE IMAGE PROCESS
      image2 = image
      gray = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
      blurred = cv2.bilateralFilter(gray, b_bf0, b_bf1, b_bf2)

      #PLAN A: threshold
      ret, thresh = cv2.threshold(blurred, MIN_VALUE, MAX_VALUE, cv2.THRESH_BINARY_INV) # options: THRESH_BINARY, THRESH_BINARY_INV, THRESH_TRUNC, THRESH_TOZERO, THRESH_TOZERO_INV

      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (b_se0, b_se1)) # options: MORPH_RECT, MORPH_ELLIPSE; Also tweak last parameter(x, x)
      morphchoice = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel) # options: MORPH_CLOSE, MORPH_OPEN, MORPH_DILATE, MORPH_ERODE
      (cnts, _) = cv2.findContours(morphchoice.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # finding_contours

      #find the biggest contour
      c = max(cnts, key = cv2.contourArea)
      
      #place c back in a list
      list_c = [ np.array( c ) ]

      #FILL OUTSIDE OF COUTOUR WITH COLOR
      stencil = np.zeros(image2.shape[:-1]).astype(np.uint8)
      cv2.fillPoly(stencil, list_c, MASK_VALUE)
      sel = stencil != MASK_VALUE # select everything that is not MASK_VALUE
      image2[sel] = FILL_COLOR # and fill it with FILL_COLOR

      # Save image to output dir
      cv2.imwrite(dir_imgs_results +'/'+ output_file_name, image2)

  print('Finished removing background for all images in folder.')

1/5 Removing BG from 81GA180035_1.jpg...
2/5 Removing BG from 81GA180035_2.jpg...
3/5 Removing BG from 81GA180035_3.jpg...
4/5 Removing BG from 81GA180035_4.jpg...
5/5 Removing BG from 81GA180035_5.jpg...
done.


### 4. Unmount Google Drive

In [4]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
